In [1]:
cd /Users/martin/Git/estates

/Users/martin/Git/estates


In [2]:
import pandas as pd
import os
import sqlalchemy
from datetime import datetime

In [19]:
folder = 'data/silver'
files = os.listdir(folder)

dataframes = []
for file in files:
    date = file.split('_')[0]
    date = datetime.strptime(date, "%Y%m%d-%H%M%S")
    
    csv = pd.read_csv(f'{folder}/{file}')
    csv = csv.assign(created_at=date)
    dataframes.append(csv)
    
df = pd.concat(dataframes, axis=0, ignore_index=True)
df = df.drop_duplicates(subset=[col for col in df.columns if col != 'created_at'])

In [20]:
df.head()

,estate_id,estate_title,estate_description_short,estate_description_long,estate_category_main_cb,estate_disposition,estate_rental_or_sell,estate_locality_district,estate_longitude,estate_latitude,...,Posudek znalce,Počet kanceláří,Anuita,Termín 1. prohlídky,Termín 2. prohlídky,estate_id.1,expires_at,Minimální kupní cena,Počet míst,Počet lůžek
0,2829839708,Prodej pozemku 2 259 m²,Pozemek 3 224 m² k prodeji Jeseník; 7 093 000 ...,Nabízíme k prodeji tři komerční pozemky v cent...,3,24,1,46,17.218935,50.217726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1101036124,Prodej stavebního pozemku 1 089 m²,"Stavební parcela 1 554 m² k prodeji Kácov, okr...","Kácov se nachází 4,7km (4 minuty) po rychlostn...",3,19,1,52,15.040132,49.763996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,420809308,"Prodej rodinného domu 685 m², pozemek 2 381 m²",Rodinný dům 685 m² k prodeji Moravské Budějovi...,Rozestavěná stavba vily Residence Vranín Na pr...,2,37,1,69,15.745152,49.050579,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2119215708,Prodej bytu 2+kk 78 m²,Byt 2+kk 78 m² k prodeji Havlíčkův Brod; 3 552...,"Havlíčkův Brod, byt 2+kk, ulice U Tunelu, po r...",1,4,1,66,15.594960,49.593576,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3934808668,Prodej bytu 2+kk 64 m²,Byt 2+kk 64 m² k prodeji Praha 8 - Bohnice; 4 ...,Nabízíme k prodeji byt 2+kk v družstevním vlas...,1,4,1,5008,14.443999,50.121771,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df.shape

(89493, 116)

In [ ]:
df.to_sql(name='silver-estates', con=engine, if_exists='replace', index=False)

# aws secrets 

In [3]:
secret_name = "estates-rds"
region_name = "eu-central-1"

In [4]:
import base64
import json
from typing import Dict

import boto3

from botocore.exceptions import ClientError

In [5]:
def get_secret(secret_name: str, region_name: str) -> Dict:
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name,
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            print("The requested secret " + secret_name + " was not found")
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            print("The request was invalid due to:", e)
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            print("The request had invalid params:", e)
        elif e.response['Error']['Code'] == 'DecryptionFailure':
            print("The requested secret can't be decrypted using the provided KMS key:", e)
        elif e.response['Error']['Code'] == 'InternalServiceError':
            print("An error occurred on service side:", e)
    else:
        # Secrets Manager decrypts the secret value using the associated KMS CMK
        # Depending on whether the secret was a string or binary, only one of these fields will be populated
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
        else:
            secret = base64.b64decode(get_secret_value_response['SecretBinary'])

        return json.loads(secret)  # returns the secret as dictionary

In [6]:
secret = get_secret(secret_name, region_name)

In [7]:
engine = sqlalchemy.create_engine("postgresql+psycopg2://{username}:{password}@{host}:{port}".format(**secret))

In [8]:
engine.table_names()

<ipython-input-8-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['silver-estates']

In [ ]:
engine.execute('select estate_id, created_at from estates where created_at')

In [ ]:
engine.table_names()

In [ ]:
df.dtypes.head(20)

In [15]:
q = """
SELECT *
FROM "silver-estates"
WHERE created_at = '2021-07-09'::date
"""
df = pd.read_sql_query(q, con=engine)

In [18]:
df.dtypes

estate_id                    int64
estate_title                object
estate_description_short    object
estate_description_long     object
estate_category_main_cb      int64
                             ...  
estate_id.1                 object
expires_at                  object
Minimální kupní cena        object
Počet míst                  object
Počet lůžek                 object
Length: 116, dtype: object

In [9]:
df_aws = pd.read_sql_table('silver-estates', con=engine)

In [10]:
df_aws.shape

(89743, 116)

In [11]:
df_aws.isna().sum()

estate_id                       0
estate_title                    0
estate_description_short        0
estate_description_long         0
estate_category_main_cb         0
                            ...  
estate_id.1                 89380
expires_at                  89380
Minimální kupní cena        89720
Počet míst                  89739
Počet lůžek                 89740
Length: 116, dtype: int64

In [12]:
col_subset = df_aws.columns[:30]

In [14]:
df_aws.loc[:, col_subset].head()

,estate_id,estate_title,estate_description_short,estate_description_long,estate_category_main_cb,estate_disposition,estate_rental_or_sell,estate_locality_district,estate_longitude,estate_latitude,...,Cukrárna,Lékárna,Školka,Lékař,Kino,Hřiště,Pošta,Restaurace,Škola,Obchod
0,2829839708,Prodej pozemku 2 259 m²,Pozemek 3 224 m² k prodeji Jeseník; 7 093 000 ...,Nabízíme k prodeji tři komerční pozemky v cent...,3,24,1,46,17.218935,50.217726,...,131.0,352.0,379.0,153.0,553.0,421.0,473.0,148.0,163.0,190.0
1,1101036124,Prodej stavebního pozemku 1 089 m²,"Stavební parcela 1 554 m² k prodeji Kácov, okr...","Kácov se nachází 4,7km (4 minuty) po rychlostn...",3,19,1,52,15.040132,49.763996,...,NaN,96.0,328.0,263.0,NaN,356.0,263.0,238.0,328.0,NaN
2,420809308,"Prodej rodinného domu 685 m², pozemek 2 381 m²",Rodinný dům 685 m² k prodeji Moravské Budějovi...,Rozestavěná stavba vily Residence Vranín Na pr...,2,37,1,69,15.745152,49.050579,...,NaN,NaN,2436.0,2927.0,NaN,2563.0,2700.0,2998.0,2469.0,NaN
3,2119215708,Prodej bytu 2+kk 78 m²,Byt 2+kk 78 m² k prodeji Havlíčkův Brod; 3 552...,"Havlíčkův Brod, byt 2+kk, ulice U Tunelu, po r...",1,4,1,66,15.594960,49.593576,...,1035.0,1263.0,1500.0,1137.0,1742.0,499.0,1067.0,995.0,1622.0,1302.0
4,385931868,Pronájem bytu 2+1 55 m²,"Byt 2+1 55 m² k pronájmu Čs. exilu, Ostrava - ...","Pronájem bytové jednotky 2 + 1, ul. Čs. exilu ...",1,5,2,65,18.169058,49.827350,...,254.0,222.0,266.0,254.0,4338.0,317.0,490.0,241.0,192.0,1324.0
